In [1265]:
import pandas_datareader as pdr
import datetime
import numpy as np 
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import time
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
import pytz
from yahooquery import Ticker

In [1112]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

# aux functions

In [1113]:
# def moving_average(array, n):
#     new_list = []
    
#     for index in range(array.shape[0]-n+1):
#         element = np.mean(array[index:index+n])
#         new_list.append(element)
    
#     return new_list

In [1114]:
def get_slope_one_unit(my_list):

    slope_list = []

    for index in range(len(my_list)-1):
        slope = (my_list[index+1]-my_list[index])

        slope_list.append(slope)
        
    return slope_list

In [1115]:
def get_trend(slope_list):

    slope_positive_list = []
    slope_negative_list = []

    count_positive = 0
    count_negative = 0

    for index in range(len(slope_list)):

        if slope_list[index] < 0:
            count_negative += 1
            count_positive = 0

        elif slope_list[index] > 0:
            count_positive += 1
            count_negative = 0

        slope_negative_list.append(count_negative)
        slope_positive_list.append(count_positive)
        
    return slope_negative_list

In [1383]:
def buy_order(average, slope_negative_list, downward_trend_wanted = 5):
    
    list_buy_masks = []
    buy_top_bottom_price_tuple_list = []
    list_percentage_drop = []
       
    for index in range(len(slope_negative_list)-1):
        observed_downward_trend = slope_negative_list[index] - slope_negative_list[index+1]
        
        if  observed_downward_trend > 2 and observed_downward_trend <= downward_trend_wanted and observed_downward_trend >= 1:
            price_at_top = average[index-observed_downward_trend+1]
            price_at_bottom = average[index+1]
            price_at_purchase = average[index+2]
            percentage_drop = 100-(price_at_bottom*100/price_at_top)
            
            if percentage_drop > 2:
                buy_condition = (price_at_purchase-price_at_bottom)/(price_at_top-price_at_bottom)
                if  buy_condition > 0.2 and buy_condition < 0.6: 
                    list_buy_masks.append(1)
                    buy_top_bottom_price_tuple = (price_at_top, price_at_bottom, price_at_purchase)
                    buy_top_bottom_price_tuple_list.append(buy_top_bottom_price_tuple)
                    list_percentage_drop.append(percentage_drop)
                else:
                    list_buy_masks.append(0)
            else:
                list_buy_masks.append(0)
                
        else:
            list_buy_masks.append(0)
            
    return list_buy_masks, buy_top_bottom_price_tuple_list, list_percentage_drop

In [1349]:
def sell_order(average, list_buy_masks, buy_top_bottom_price_tuple_list, list_percentage_drop):
    
    list_sell_indices = []
    buy_list = np.where(np.asarray(list_buy_masks) == 1)[0]
    
    out_of_bounds = False
    tuple_counter = 0

    for start_index in buy_list:
        top_price = buy_top_bottom_price_tuple_list[tuple_counter][0]
        bottom_price = buy_top_bottom_price_tuple_list[tuple_counter][1]
        purchase_price = buy_top_bottom_price_tuple_list[tuple_counter][2]
        percentage_drop = list_percentage_drop[tuple_counter]
        trailing_percentage = (1-percentage_drop/100)
        
        count = 1
        
        try:
            while average[start_index+count+2] > purchase_price*(1-percentage_drop/100) and average[start_index+count+2] < 0.9*top_price+purchase_price*0.1: 
                count += 1
            list_sell_indices.append(start_index+count)
        except IndexError:
            buy_list = np.delete(buy_list, np.where(buy_list == start_index)[0][0])
            out_of_bounds = True

        tuple_counter += 1
    
    if buy_list != []:
        return list_sell_indices, buy_list, trailing_percentage, top_price
    
    else:
        return list_sell_indices, buy_list, 0, 0

# get a list of companies from the US stock market

In [1273]:
def fetch_traded_volumes(soup):

    soup_string = str(soup)

    list_actions = ["Sell</span>", "Buy</span>", "Strong Buy</span>", "Strong Sell</span>", "Neutral</span>"]

    for action in list_actions:
        soup_string = soup_string.replace(action, 'cantankerous')
    soup_string_split = soup_string.split("cantankerous")
    soup_string_split = soup_string_split[1:]

    list_traded_volumes = []

    for sublist in soup_string_split:
        volume_string = sublist.split("</td>")[1].split(">")[-1]
        if volume_string[-1] == "B":
            volume_number = float(volume_string[:-1])*np.power(10,9)
        elif volume_string[-1] == "M":
            volume_number = float(volume_string[:-1])*np.power(10,6)
        elif volume_string[-1] == "K":
            volume_number = float(volume_string[:-1])*np.power(10,3)
        else:
            volume_number = float(volume_string)

        list_traded_volumes.append(volume_number)
        
    return list_traded_volumes

In [1276]:
#r = requests.get("https://www.tradingview.com/markets/stocks-usa/market-movers-most-volatile/")
r = requests.get("https://www.tradingview.com/markets/stocks-usa/market-movers-active/")
html_doc = r.text
soup = BeautifulSoup(html_doc , 'html.parser')
soup_tag = soup.find_all("a", href=True)

list_traded_volumes = fetch_traded_volumes(soup)

company_symbols_list = []
count = 0
for a in soup_tag:
    string_a = str(a)
    if "symbols" in string_a and "target" in string_a:
        symbol = string_a.split("<")[-2].split(">")[-1]
        if symbol not in ["EROS", "WLL"]:
            if list_traded_volumes[count] > 5*np.power(10, 7):
                company_symbols_list.append(symbol)
        count += 1

In [1299]:
company_symbols_list

['OAS']

In [1227]:
######################################

In [1297]:
# using alpha vantage API

def vantage_fetch_latest_value(company_symbols_list):

    list_data = []
    for company in company_symbols_list:
        try:

            print(company)
            ts = TimeSeries(key='XQJSCNK7QEMWH6RZ', output_format='pandas')
            data, meta_data = ts.get_intraday(symbol=company, interval='1min', outputsize='full')
            data = data["4. close"]
            list_data.append(data)
            print(meta_data)
            
        except ValueError:
            print("company symbol " + company + " not a valid API call")

    return list_data, meta_data

In [1330]:
# using yahoo query API

def yahooquery_fetch_latest_value(company_symbols_list):
    
    for company in company_symbols_list:
        try:
            tickers = Ticker(company)
            dic_data = tickers.financial_data
            data = dic_data["OAS"]["currentPrice"]

        except ValueError:
            print("company symbol " + company + " not a valid API call")

    return data

# quick stock analysis (overal trend and mean price)

In [1293]:
list_expensive_stocks = []
list_expensive_symbols = []
count = 0

for average in list_data_company:
    company_symbol = company_symbols_list[count]
    mean = np.mean(average)
    
    if mean < 1:
        if average[-1] - average[0] > 0:
            list_expensive_stocks.append(average)
            list_expensive_symbols.append(company_symbol)
    count += 1

In [1294]:
list_expensive_symbols

['OAS']

In [1295]:
# compare to just holding the position from start to end

list_profit_holding = []

for average in list_expensive_stocks:
    
    hold_stock_profit = average[-1] - average[0]
    list_profit_holding.append(hold_stock_profit)

print(str(np.sum(np.asarray(list_profit_holding) > 0)) + "/" + str(len(list_profit_holding)))
print(np.sum(list_profit_holding))

1/1
0.43040000000000006


# test the returns

In [1384]:
for n_down in range(1, 20):
    print("WE WAIT FOR PRICES TO GO DOWN FOR AT MOST " + str(n_down) + " MINUTES IN A ROW")
        
    overall_profit = []
    number_positive_trades = []
    number_negative_trades = []
    list_value_buys = []

    #for average in list_expensive_stocks:
    for average in list_data_company:

        slope_list = get_slope_one_unit(average)
        slope_negative_list = get_trend(slope_list)
        list_buy_masks, buy_top_bottom_price_tuple_list, list_percentage_drop = buy_order(average, slope_negative_list, n_down)
        list_sell_indices, list_buy_indices, _, _ = sell_order(average, list_buy_masks, buy_top_bottom_price_tuple_list, list_percentage_drop)
            
        list_profit = []

        for index in range(len(list_sell_indices)):
            list_value_buys.append(average[list_buy_indices[index]+2])
            profit = (average[list_sell_indices[index]+2]-average[list_buy_indices[index]+2])

            list_profit.append(profit)

        overall_profit.append(np.sum(list_profit))
        number_positive_trades.append(np.sum(np.asarray(list_profit) > 0))
        number_negative_trades.append(np.sum(np.asarray(list_profit) < 0))

    print("total invested: " + str(np.sum(list_value_buys)))
    print("% profit: " + str(((np.sum(list_value_buys)+np.sum(overall_profit))*100/np.sum(list_value_buys))-100))
    print("total profit: " + str(np.sum(overall_profit)))
    print("average profit per trade: " + str(np.sum(overall_profit)/(np.sum(number_positive_trades)+np.sum(number_negative_trades))))
    print("total winning trades: " + str(np.sum(number_positive_trades)) + " total losing trades: " + str(np.sum(number_negative_trades)))

WE WAIT FOR PRICES TO GO DOWN FOR AT MOST 1 MINUTES IN A ROW
total invested: 0.0
% profit: nan
total profit: 0.0
average profit per trade: nan
total winning trades: 0 total losing trades: 0
WE WAIT FOR PRICES TO GO DOWN FOR AT MOST 2 MINUTES IN A ROW
total invested: 0.0
% profit: nan
total profit: 0.0
average profit per trade: nan
total winning trades: 0 total losing trades: 0
WE WAIT FOR PRICES TO GO DOWN FOR AT MOST 3 MINUTES IN A ROW
total invested: 4.4117
% profit: 2.2281660131015286
total profit: 0.09830000000000005
average profit per trade: 0.010922222222222229
total winning trades: 8 total losing trades: 1
WE WAIT FOR PRICES TO GO DOWN FOR AT MOST 4 MINUTES IN A ROW
total invested: 6.594999999999999
% profit: 0.8870356330553477
total profit: 0.05850000000000005
average profit per trade: 0.004500000000000004
total winning trades: 10 total losing trades: 3
WE WAIT FOR PRICES TO GO DOWN FOR AT MOST 5 MINUTES IN A ROW


/home/thomas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/home/thomas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars
/home/thomas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars
/home/thomas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


total invested: 6.594999999999999
% profit: 0.8870356330553477
total profit: 0.05850000000000005
average profit per trade: 0.004500000000000004
total winning trades: 10 total losing trades: 3
WE WAIT FOR PRICES TO GO DOWN FOR AT MOST 6 MINUTES IN A ROW
total invested: 6.594999999999999
% profit: 0.8870356330553477
total profit: 0.05850000000000005
average profit per trade: 0.004500000000000004
total winning trades: 10 total losing trades: 3
WE WAIT FOR PRICES TO GO DOWN FOR AT MOST 7 MINUTES IN A ROW
total invested: 6.594999999999999
% profit: 0.8870356330553477
total profit: 0.05850000000000005
average profit per trade: 0.004500000000000004
total winning trades: 10 total losing trades: 3
WE WAIT FOR PRICES TO GO DOWN FOR AT MOST 8 MINUTES IN A ROW
total invested: 6.594999999999999
% profit: 0.8870356330553477
total profit: 0.05850000000000005
average profit per trade: 0.004500000000000004
total winning trades: 10 total losing trades: 3
WE WAIT FOR PRICES TO GO DOWN FOR AT MOST 9 MINUT

# IBPy library stuff and real time data stream decisions

In [789]:
from ibapi import wrapper
from ibapi.client import EClient
from ibapi.utils import iswrapper #just for decorator
from ibapi.common import *

In [791]:
from ib.ext.Contract import Contract
from ib.ext.Order import Order
from ib.ext import EWrapper
from ib.opt import Connection, message
import ib

In [792]:
def error_handler(msg):
    """Handles the capturing of error messages"""
    print("Server Error: %s" % msg)

def reply_handler(msg):
    """Handles of server replies"""
    print("Server Response: %s, %s" % (msg.typeName, msg))

In [793]:
# class TestApp(wrapper.EWrapper, EClient):
#     def __init__(self, id_order):
#         wrapper.EWrapper.__init__(self)
#         EClient.__init__(self, wrapper=self)
#         self.id_order = id_order

#     @iswrapper
#     def nextValidId(self, orderId:int):
#         print("setting nextValidOrderId:", orderId)
#         self.nextValidOrderId = orderId
#         self.id_order = orderId
#         # here is where you start using api
#         self.reqAccountSummary(9002, "All", "$LEDGER")
#         self.disconnect()
     
#     def get_id(self):
#         order_id = self.id_order
#         return order_id
    
# app = TestApp(0)
# app.connect("192.168.2.47", 7497, clientId=1260)
# app.run()
# order_id = app.get_id()
# print(order_id)
    

In [794]:
# make the contract

def create_contract(symbol, sec_type, exch, prim_exch, curr):
    
    contract = Contract()
    contract.m_symbol = symbol
    contract.m_secType = sec_type
    contract.m_exchange = exch
    contract.m_primaryExch = prim_exch
    contract.m_currency = curr
    
    return contract

In [895]:
# define 3 types of orders, to make the overall buy limit, sell limit (profit taker) and trailing stop

def make_market_order(action, quantity, order_id, transmit, tif):
    
    order = Order()
    order.m_orderType = 'MKT'
    order.m_totalQuantity = quantity
    order.m_action = action
    order.m_orderId = order_id
    order.m_transmit = transmit
    order.m_tif = tif
    
    return order

def make_limit_order(action, quantity, price, order_id, transmit, tif = None, parent_id = None):

    order = Order()
    order.m_orderType = 'LMT'
    order.m_totalQuantity = quantity
    order.m_action = action
    order.m_lmtPrice = price
    order.m_orderId = order_id
    order.m_transmit = transmit
    
    if tif != None:
        order.m_tif = tif
    
    if parent_id != None:
        order.m_parentId = parent_id

    return order

def make_stop_order(action, quantity, order_id, transmit, trail_percentage, parent_id):

    order = Order()
    order.m_orderType = 'TRAIL'
    order.m_totalQuantity = quantity
    order.m_action = action
    order.m_orderId = order_id
    order.m_transmit = transmit
    order.m_trailingPercent = trail_percentage
    order.m_parentId = parent_id
    
    return order

In [1366]:
if __name__ == "__main__":
    
    time_series_stock = []

    n_down = 5
    market_opens = True
    minute_counter = 0
    order_id = 1

    while market_opens:

        t1 = time.time()

        latest_price = yahooquery_fetch_latest_value(["OAS"])
        time_series_stock.append(latest_price)
        print(latest_price)

        minute_counter += 1

        slope_list = get_slope_one_unit(time_series_stock)
        slope_negative_list = get_trend(slope_list)
        list_buy_masks, buy_top_bottom_price_tuple_list, list_percentage_drop = buy_order(time_series_stock, slope_negative_list, n_down)
        list_sell_indices, list_buy_indices, trailing_percentage, top_price = sell_order(time_series_stock, list_buy_masks, buy_top_bottom_price_tuple_list, list_percentage_drop)
        print(list_buy_masks)
        
        tz_NY = pytz.timezone('America/New_York') 
        datetime_NY = datetime.now(tz_NY)
        formatted_time = datetime_NY.strftime("%H:%M:%S")
        market_opens = datetime.strptime(formatted_time, "%H:%M:%S") < datetime(1900, 1, 1, 16, 15, 0)

        t2 = time.time()
        tick_time = t2 - t1

        if tick_time > 60:
            pass
        else:
            time.sleep(60-tick_time)

        if list_buy_masks != [] and list_buy_masks[-1] == 1:
            
            tws_conn = Connection.create("<ip>", port=7497, clientId=<id>)
            tws_conn.connect()
            tws_conn.register(error_handler, 'Error')
            tws_conn.registerAll(reply_handler)

            quantity = 17
            order_id = order_id

            my_contract = create_contract('OAS', 'STK', 'SMART', 'SMART', 'USD')

            # limit buy order
            my_order_buy = make_limit_order("BUY", quantity, latest_price, order_id, False, "DAY", None)
            tws_conn.placeOrder(order_id, my_contract, my_order_buy)

            # market buy order
            #my_order_buy = make_market_order("BUY", quantity, order_id, False, "IOC")
            #tws_conn.placeOrder(order_id, my_contract, my_order_buy)

            # limit order sell
            my_order_sell = make_limit_order("SELL", quantity, top_price, order_id+1, False, None, order_id)
            tws_conn.placeOrder(order_id+1, my_contract, my_order_sell)

            # trail stop
            my_order_sell = make_stop_order("SELL", quantity, order_id+2, True, trailing_percentage, order_id)
            tws_conn.placeOrder(order_id+2, my_contract, my_order_sell)
            
            time.sleep(2)
            order_id += 3
            
            tws_conn.disconnect()

0.6239
[]


/home/thomas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


0.6362
[]
0.6319
[0]
0.6342
[0, 0]
0.6251
[0, 0, 0]
0.6251
[0, 0, 0, 0]
0.6322
[0, 0, 0, 0, 0]
0.6349
[0, 0, 0, 0, 0, 0]
0.6318
[0, 0, 0, 0, 0, 0, 0]
0.624
[0, 0, 0, 0, 0, 0, 0, 0]
0.6254
[0, 0, 0, 0, 0, 0, 0, 0, 0]
0.6239
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.6278
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.6292
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.6273
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.629
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.629
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.627
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.6261
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.6189
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.6191
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.6189
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.6102
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.6058
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.6061
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

0.6137
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.6137
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
